<a href="https://colab.research.google.com/github/akshatasangale/Akshata_/blob/main/Ml_project1_Movie__Recommendation__System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
credits_df =pd.read_csv("credits.csv")
movies_df = pd.read_csv("movies.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'credits.csv'

In [ ]:
credits_df

In [ ]:
movies_df

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)


In [ ]:
credits_df

In [ ]:
movies_df

In [ ]:
credits_df.head()

In [ ]:
movies_df.tail()

In [ ]:
#from pandas.core.reshape import merge
movies_df = movies_df.merge(credits_df,on='title')

In [ ]:
movies_df.shape

In [ ]:
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
movies_df = movies_df[['movie_id_x','title','overview','genres','keywords','cast_x','crew_x']]

In [ ]:
movies_df.head()

In [ ]:
movies_df.info()

In [ ]:
movies_df.isnull().sum()


In [ ]:
movies_df.dropna(inplace=True)

In [ ]:
movies_df.duplicated().sum()


In [ ]:
movies_df.duplicated()

In [ ]:
movies_df.iloc[0].genres


In [ ]:
import ast

In [ ]:
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)
movies_df.head()

In [ ]:
def convert3(obj):
  L=[]
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      L.append(i['name'])
      counter +=1
    else:
      break
    return L

In [ ]:
 movies_df['cast_x'] = movies_df['cast_x'].apply(convert3)

In [ ]:
movies_df.head()

In [ ]:
def featch_dirctor(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      L.append(i['name'])
    return L

In [ ]:
movies_df['crew_x']=movies_df['crew_x'].apply(featch_dirctor)

In [ ]:
movies_df.head()

In [ ]:
movies_df['overview'][0]

In [ ]:
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())

In [ ]:
movies_df.head()

In [ ]:
movies_df['genres'] = movies_df['overview'].apply(lambda x:[i.replace(" ","") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])


In [ ]:
#movies_df['cast_x'] = movies_df['cast_x'].apply(lambda x:[i.replace(" ","") for i in x])
#movies_df['crew_x'] = movies_df['crew_x'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies_df['cast_x'] = movies_df['cast_x'].apply(lambda x: [i.replace(" ", "") for i in x] if x is not None else [])
movies_df['crew_x'] = movies_df['crew_x'].apply(lambda x: [i.replace(" ", "") for i in x] if x is not None else [])


In [ ]:
movies_df.head()

In [ ]:
movies_df['tags'] = movies_df['overview']+movies_df['genres']+movies_df['keywords']+movies_df['cast_x']+movies_df['crew_x']

In [ ]:
movies_df.head()

In [ ]:
new_df = movies_df [['movie_id_x', 'title', 'tags']]

In [ ]:
new_df

In [ ]:
new_df['tags']= new_df['tags'].apply(lambda x:' '.join(x))
#new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x.split()))
#new_df['tags'] = new_df['tags'].apply(lambda x: x.split())


In [ ]:
new_df

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']= new_df['tags'].apply(lambda X:X.lower())

In [ ]:
new_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000 ,stop_words = 'english')

In [ ]:
cv.fit_transform(new_df['tags']).toarray().shape

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
from nltk.stem import PorterStemmer
import pandas as pd

# Assuming new_df is your DataFrame and 'tags' is the column you want to apply stemming to
ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

new_df['tags'] = new_df['tags'].apply(stem)


In [ ]:
 from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

In [ ]:
cosine_similarity(vectors).shape

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0].shape

In [ ]:
sorted(list(enumerate(similarity[0])), reverse = True , key = lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]]['title'])


In [ ]:
recommend('Avatar') #test

In [ ]:
recommend('Iron Man') #test

In [ ]:
recommend('The Dark Knight Rises') #test